# Vocab Analysis 
## Section 2: Prepare the Data

### 1. Import libraries

In [72]:
import pandas as pd
import sqlite3
import json
from datetime import datetime, timedelta, date
import time

### 2. Import Data

In [73]:
location = "datasets/collection.anki2"
cnx = sqlite3.connect(location) # create sql file connection

In [74]:
# TDD backbone assertion to confirm a function call returns the desired result
def assertEquals(actual, expected, desc):
    assert(actual==expected), desc + " result: " + str(actual) + ", expected: " + str(expected)
    return "OK"

### A word anout how the data was stored

Anki, the Open Source, Spaced Repetition software (& app & service), saves a student's data in a few locations. There are the "Notes" which are the raw info used to make cards (fields of vocab data, metatags, trivia facts, images, audio, etc..) Then there are the "Cards" which are the actual studied items, where the study overview data is stored (such as study date-times, repetitions (reviews), intervals (how long a card is to be remembered), lapses (forgets & subsequent interval resets)). Additionally, data concerning the entire collection is stored under something cryptically called "Columns". Lastly, there is a "RevLog" which contains all the study data in detail for each individual repetition (study datetime, card studied, etc..) This document was critical to piecing together the puzzle: https://github.com/ankidroid/Anki-Android/wiki/Database-Structure

### 3. Extract Deck Creation Date

In [75]:
df_c = pd.read_sql_query("SELECT * FROM col", cnx)
crt = df_c['crt'][0] # save collection creation date (in epoch time)
pd_crt = pd.to_datetime(crt, unit = 's')
print(pd_crt)

assertEquals(str(pd_crt), "2013-01-08 09:00:00", "Collection Creation Date")

2013-01-08 09:00:00


'OK'

### 4. Extract field names to label columns

In [76]:
field_names = []
for row_index, blob in df_c['models'].items():
    for model_id, data in json.loads(blob).items():
        field_names += list(map(lambda fld: fld['name'], data['flds']))
field_names.append('Tags')
expected_names = ['Term', 'Yomi1', 'Translation', 'Translation2', 'Translation3', 'AlternateForms', 'PartOfSpeech', 'Sound', 'Sound2', 'Sound3', 'Examples', 'ExamplesAudio', 'AtoQ', 'AtoQaudio', 'AtoQkana', 'AtoQtranslation', 'QandApicture', 'answerPicture', 'Meaning1', 'SimilarWords', 'RelatedWords', 'Breakdown1', 'Comparison', 'Usage', 'Prompt1', 'Prompt2', 'KakuMCD', 'IuMCD', 'ExtraMemo', 'Yomi2', 'Meaning2', 'Breakdown2', 'Picture1', 'Picture2', 'Picture3', 'Picture4', 'HinshiMarker', 'Hint', 'Term2', 'ArabicNumeral', 'CounterKanji', 'Mnemonic', 'SameSoundWords', 'Yomi3', 'gChap', 'gBook', 'semester', 'gNumber', 'Transliteration', 'SoloLookCards', 'TagOverflow', 'blank1', 'blank2', 'Tags']

In [77]:
assertEquals(field_names, expected_names, "Field Names")

'OK'

### 5. Import card study data into data frame "df_cards"

In [78]:
# Step 6: Take in study data from Anki collection
df_cards = pd.read_sql_query("SELECT * FROM cards", cnx)
assertEquals(df_cards.shape[0],19315,"Rows")#6386, 21979, 19363
assertEquals(df_cards.shape[1],18,"Columns")

'OK'

### 6. Confirm that card data model matches expected format

In [79]:
expected_columns_1 = ['id', 'nid', 'did', 'ord', 'mod', 'usn', 'type', 'queue', 'due', 'ivl', 'factor',
 'reps', 'lapses', 'left', 'odue', 'odid', 'flags', 'data']

def lists_equal(a,b):
    return (a == b).all()

assertEquals(lists_equal(df_cards.columns.values, expected_columns_1), True, "Card Columns Import")

'OK'

### 7. Shallow check for duplicates (matching rows)

In [80]:
 def has_dupes(df_in):
    dupe = df_in.duplicated()
    return df_in.loc[dupe].shape[0] != 0

In [81]:
assertEquals(has_dupes(df_cards), False, "Duplicates Not Found")

'OK'

### 8. Remove unneeded card dataframe columns

In [82]:
def print_line_break():
    print("-"*75)

In [83]:
def print_before_after(b, a, t=""):
    if t != "":
        print_line_break()
        print(t)
    print_line_break()
    print("Before: " + str(b))
    print_line_break()
    print("After: " + str(a))
    print_line_break()

In [84]:
df_cards_001_less_cols = df_cards.copy()
df_cards_001_less_cols = df_cards_001_less_cols.drop(['did','usn','type','mod','left','odue','odid','flags','data'],axis=1)
expected_columns_2 = ['id', 'nid', 'ord', 'queue', 'due', 'ivl', 'factor', 'reps','lapses']

print_before_after(df_cards.columns.values, df_cards_001_less_cols.columns.values,"Card Columns:")

assertEquals(lists_equal(df_cards_001_less_cols.columns.values, expected_columns_2), True, "Card Model Slimmed")

---------------------------------------------------------------------------
Card Columns:
---------------------------------------------------------------------------
Before: ['id' 'nid' 'did' 'ord' 'mod' 'usn' 'type' 'queue' 'due' 'ivl' 'factor'
 'reps' 'lapses' 'left' 'odue' 'odid' 'flags' 'data']
---------------------------------------------------------------------------
After: ['id' 'nid' 'ord' 'queue' 'due' 'ivl' 'factor' 'reps' 'lapses']
---------------------------------------------------------------------------


'OK'

### 9. Import notes (words) into data frame "df_notes"

In [85]:
# let's take in the 'notes' table, and explicitly save the note id ("nid") 
df_notes = pd.read_sql_query("SELECT * FROM notes", cnx)
df_notes = df_notes.rename(columns={'id':'nid'})

In [86]:
assertEquals(df_notes.shape[0],8384,"Rows") # 2791, 9784, 8403
assertEquals(df_notes.shape[1],11,"Columns")

'OK'

### 10. Remove (drop) unneeded fields (columns)

In [87]:
df_notes_old_col_vals = df_notes.columns.values
df_notes = df_notes.drop(['guid','mid','usn','sfld','csum','flags','data'],axis=1)
#print(df_notes.columns.values)
print_before_after(df_notes_old_col_vals, df_notes.columns.values)

---------------------------------------------------------------------------
Before: ['nid' 'guid' 'mid' 'mod' 'usn' 'tags' 'flds' 'sfld' 'csum' 'flags' 'data']
---------------------------------------------------------------------------
After: ['nid' 'mod' 'tags' 'flds']
---------------------------------------------------------------------------


### 11. Split "fields" column into multiple, assign field names, drop combined col

In [88]:
def time_it(func, *args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    end = time.time()
    # https://stackoverflow.com/questions/8885663/how-to-format-a-floating-number-to-fixed-width-in-python
    print("{:.0f}".format((end - start)*1000) + " miliseconds")

In [89]:
for i in range(0,len(expected_names)-1):
    df_notes[expected_names[i]] = df_notes.flds.str.split('').str.get(i)
assertEquals('flds' in df_notes.columns.values, True, "'flds' Column Found")
df_notes = df_notes.drop(['flds'],axis=1)
assertEquals('flds' not in df_notes.columns.values, True, "'flds' Column Not Found")
print(df_notes.columns.values)

['nid' 'mod' 'tags' 'Term' 'Yomi1' 'Translation' 'Translation2'
 'Translation3' 'AlternateForms' 'PartOfSpeech' 'Sound' 'Sound2' 'Sound3'
 'Examples' 'ExamplesAudio' 'AtoQ' 'AtoQaudio' 'AtoQkana'
 'AtoQtranslation' 'QandApicture' 'answerPicture' 'Meaning1'
 'SimilarWords' 'RelatedWords' 'Breakdown1' 'Comparison' 'Usage' 'Prompt1'
 'Prompt2' 'KakuMCD' 'IuMCD' 'ExtraMemo' 'Yomi2' 'Meaning2' 'Breakdown2'
 'Picture1' 'Picture2' 'Picture3' 'Picture4' 'HinshiMarker' 'Hint' 'Term2'
 'ArabicNumeral' 'CounterKanji' 'Mnemonic' 'SameSoundWords' 'Yomi3'
 'gChap' 'gBook' 'semester' 'gNumber' 'Transliteration' 'SoloLookCards'
 'TagOverflow' 'blank1' 'blank2']


### 12. Check notes for duplicates (shallow check)

In [90]:
assertEquals(has_dupes(df_notes), False, "Duplicates Not Found")

'OK'

### 13. Check for duplicates by term field in notes data frame

In [91]:
def has_dupe_terms(df_in):
    location = df_in['Term'].duplicated()
    return df_in.loc[location].shape[0] != 0

In [92]:
assertEquals(has_dupe_terms(df_notes), False, "Duplicates Found")

'OK'

### 14. Confirm that duplicates dataframe is empty (no dups exist)

In [93]:
dupe = df_notes['Term'].duplicated() #creates list of True/False values
print(df_notes[dupe].shape)
assertEquals(df_notes[dupe].shape[0], 0, "Duplicates dataframe is empty.")

(0, 56)


'OK'

### 15. Inspect an individual card by its term

In [94]:
# Postal service
def inspect_note(df_in, term):
    return df_in[df_in['Term']==term]

In [95]:
sel1 = inspect_note(df_notes,'郵便')
sel1

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2
3282,1361674609381,1555887839,Japanese Marked abaCheckNuance addDefinition ...,郵便,ゆうびん,"mail, postal service",,,〒,"Common word, Noun",...,,,,,,,,,,


### Save Point, Commit, Bonfire (for you Souls fans)*

At the point in time of the data extraction where the (meta) tag information is made available, we can treat it to both clarify (rename poorly worded tags) & reduce (delete unneeded tags). Since we now have all fields split into their own columns as well, we can treat (modifiy & improve) the columns as well, in a 1-2 process: (1) Fix the tags & (2) Fix the columns
*https://en.wikipedia.org/wiki/Souls_(series)

In [96]:
def shorten_list(takeIn, takeOut):
    temp = takeIn.lower().split() # split all the words into a list
    temp2 = [word for word in temp if word.lower() not in takeOut] # create a shorter list of words minus the take-outs
    return ' '.join(temp2) # return that shorter list as a string

In [97]:
tag_remove_list = ['japanese', 'checkpicture', 'complete', 'haspicture', 'nomemo',
                   'researched', 'aaaeditthis', 'addaudio', 'addaudio2', 'addaudioNow',
                   'addmore','adjustformatting', 'hascomparison', 'hasmnemonic',
                   'customediting','wikidefinition', 'givewill','addaudionow','addprompt',
                   'checknuance','giveyaneury','hastextimage', 'marked', 'addpicture',
                   'addexampletranslation','basicnumeric', 'genkiplus', 'hasaudio',
                   'nativeaudio', 'adddefinition','addexamples', 'addjapaneseprompt',
                   'computervoice','haspoliteprefix','nongoo','customdefinition','hashint',
                   'abahipriorityfix','kaki','mcd','nobodyknows+','missingwordtype',
                   'image','duplicate', 'hasprompt', 'ninshiki','abachecknuance',
                   'hasflag','things', 'jim', 'hasunicode', 'editthis','aaahipriority',
                   'hassimpledef', 'givecodie', 'forjimmy', 'hasnativeaudio', 'givejimmy2',
                   'checkaudio', 'checkwriting', 'hasjlptlevel', 'makekaki', 'checknuance2',
                   'checkagain', 'newaudio', 'mail', 'checkexamples','elementaryschool',
                   'nvc', 'checkprompt', 'gavejimmy', 'addnativeaudio','checkreading',
                   'givecodieapril', 'activated', 'fixformatting','hasplacesuffix',
                   'hassuffix','addtranslation','addnewcardtype','addnuance','addtextimage',
                   'semicomplete', 'removeroboaudio','fixaudio','hasgramconj', 
                   'hasquestion', 'addkanji','changenotetype', 'famous', 'challenging',
                   'kuverb', 'givwill','karutapoems', 'map', 'hasvisualcomparison',
                   'picturekaki', 'jyugemu', '2018', 'type1', 'hasslang', 'apologies',
                   'month', 'definitionresearched','soundshift', 'basics1', 'tsuverb',
                   'facebook', 'uverb', 'checkfrequency', 'degree', 'hasdefinition',
                   'addtransliteration', 'dnd', 'introductions', 'adjustprompt',
                   'job', 'particle', 'services', 'mature', 'splitpictures', 
                   'egaki', 'type5k', 'intimate','extrainfo', 'irregular', 'unlisted',
                   'fromwiki', 'checkdifference','addpronunciationdiagram', 'reset',
                   'currentevents', 'doubletextimage', 'comparison', 'verbscompoundpast2',
                   'attention', 'addmemo', 'averb', 'radio','hasascii', 'fontadjusted',
                   'haspronunciation', 'borroweddefinition','alphabet', 'graphics',
                   'chiebukuro', 'duolingo', 'ateji', 'fact','type5s', 'fixpicture',
                   'politebydefault', 'objects','sensitive', 'groupword', 'addmnemonic',
                   'hasmore', 'quote', 'checkformatting','overlap', 'kotobankdef',
                   'hasrudeness', 'changedeck', 'specialformatting','yoga',
                   'hasjapaneseprompt', 'hasprefix','questionword', 'business', 
                   'postoffice', 'firstten', 'money', 'robotvoice2', 'ichidan', 'godan',
                   'weather','count', 'nodefinition', 'muverb', 'addcomparisonchart', 
                   'ruverb', 'phone', 'conjugated','haddiv','vulgar','fromkaruta',
                   'karutamanual', 'teform', 'qanda', '2019'
                  ]

### 16. Remove unneeded tags (meta-data)

In [98]:
# survey a few notes to see example tag data
df_notes.head(3)

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2
0,1331799797110,1511481489,Japanese Marked abaCheckNuance checkNuance co...,臨機応変,りんきおうへん,adapting oneself to the requirements of the mo...,,,,"Noun, No-adjective",...,,,,,,,,,,
1,1331799797112,1511481489,Japanese complete noMemo researched wwwjdic,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",...,,,,,,,,,,
2,1331799797113,1511481489,Japanese Marked abaCheckNuance checkNuance co...,苦汁,にがり,bittern; concentrated solution of salts (esp. ...,,,,Noun,...,,,,,,,,,,


In [99]:
# likely useful tags: katakana, Waseigo, Food, Phrases, casual, restaurant, travel, commonWord, noun, suruVerb

df_notes_001_less_tags = df_notes.copy() #originally "df_notes_less_tags"
df_notes_001_less_tags['tags'] = df_notes_001_less_tags['tags'].apply(lambda x: shorten_list(str(x), tag_remove_list))

print_before_after(df_notes['tags'].iloc[0], df_notes_001_less_tags['tags'].iloc[0],"Tags for " + df_notes['Term'].iloc[0])

assertEquals("Japanese" in df_notes['tags'].iloc[0].split(), True, "Contains Tag 'Japanese'")
assertEquals("Japanese" in df_notes_001_less_tags['tags'].iloc[0].split(), False, "Contains Tag 'Japanese'")

---------------------------------------------------------------------------
Tags for 臨機応変
---------------------------------------------------------------------------
Before:  Japanese Marked abaCheckNuance checkNuance complete noMemo researched wwwjdic yojijukugo 
---------------------------------------------------------------------------
After: wwwjdic yojijukugo
---------------------------------------------------------------------------


'OK'

### 17. Rename useful tags (meta-data) that were poorly named

In [100]:
# replace list (formerly named 'tag_replace_list')
tag_rename_dict = {
    'aalowfrequency':'rare checked', 'aatechnical':'technical checked', 'aaanonkaiwa':'nonconvo checked',
    'wwwjdic':'fromdict', 'expression':'phrases', 'numberonly':'number',
    'grammarpoint':'grammar', 'jisho':'fromdict', 'pointingword':'directions',
    'geometry':'math technical', 'genki':'textbook', 'jpn202':'college',
    'jpn201':'college', 'jpn101':'college', 'jpn102':'college', 'kentei':'fromexam',
    'proficiencytest':'fromexam', 'bodypart':'body', '5kyuu':'fromexam',
    'linguisticreference':'technical', 'conversation':'convo',
    'fromconvo':'convo', 'culturepoint':'culture', 'checkednuance':'checked',
    'checkedpictures':'checked', 'checkednuance':'checked', 'medical':'technical',
    'anatomy':'body', 'places':'place', 'animals':'animal',
    'newspaperterm':'fromnewspaper', 'checkedreading':'checked',
    'abbreviation':'abbr','firstsemester':'semester1','onecharacter':'len1',
    'sentence':'phrase', 'verbs':'verb', 'convook':'checked convo','inuse':'checked',
    'nuancechecked':'checked','insects':'animal insect','sightseeing':'travel',
    'accessories':'clothing', 'grammarsuffix':'suffix', 'oceanlife':'animal ocean',
    'science':'technical', 'written':'nonconvo', 'notrare':'checked',
    'aajoke':'silly', 'intonationcompare':'hassimilar', 'ij':'textbook',
    'goodcard':'inspect','aahilevel':'challenging inspect', 'ijvocab':'textbook',
    'cliothing':'clothing','unused':'nonconvo rare checked',
    'aaunused':'nonconvo rare checked', 'samesound':'hassame','animals':'animal',
    'dictionary':'fromdict','usuallywritteninkana':'kana',
    'abVeryRare':'rare checked', 'yojijukugo':'rare idiom', 'abcasual':'casual checked convo',
    'literaryform':'nonconvo', 'onomatopoeiclike':'onomatopoeic','kenjo':'humble',
    'colors':'color', 'forest':'nature','flower':'plant nature', 'aaok':'checked',
    'questions': 'question', 'adverbs':'adverb','book2':'textbook',
    'book1':'textbook','proficiencytest':'fromtest','animalscomplete':'animal',
    'sonkei':'respectful','eating':'food','fruit':'food','neverused':'nonconvo rare',
    'domainspecific':'technical','seaons':'season','seasons':'season',
    'prefecture':'place','plantpart':'plant', "hakataben":"dialect", "fish":"animal fish",
    "transitive":"transitive verb", "intransitive":"intransitive verb",
    "aaunecessary":"nonconvo checked", "vegetables":"vegetable food plant",
    "counters":"counter", "senmonyougo":"technical", "countries":"country place",
    "date":"datesandtime", "rarelyused":"rare", "aaakaiwa":"convo checked", "cool":"inspect",
    "investigate":"inspect"
}

#todo: investigate:
#editformatting,  datesandtime, linguistics, reference, adult, adjustpicture, checkpronunciation, addhint, challenging, inspect

In [101]:
def replace_list(takeIn, replaceDict):
    temp = takeIn.lower().split()
    temp2 = []
    for word in temp:
        if word in replaceDict:
            temp2.append(replaceDict.get(word)) # if the word exists in the dictionary, replace it
        else:
            temp2.append(word) # if the word doesnt't exist in the dictionary, leave it alone
    return ' '.join(temp2) # return that shorter list as a string

# inspect further:
# multiwriting, multimeaning, multipicture, multiterm, multireading, mergeterms, checkpronunciation, customterm,
# goodcard, personalized, silly, addjlptlevel, checkpronunciation, mergeterms, customterm, transportation vs travel

# categorize: iadjective, naajective, verb, counter, commonword, suruverb, pronoun, question, phrases, kuverb, godan, ichidan, intransitive, transitive, noun, adverbialnoun

In [102]:
df_notes_002_better_tags = df_notes_001_less_tags.copy() # originally "df_notes_better_tags"
df_notes_002_better_tags['tags'] = df_notes_002_better_tags['tags'].apply(lambda x: replace_list(str(x), tag_rename_dict))

print_before_after(df_notes_001_less_tags['tags'].iloc[0], df_notes_002_better_tags['tags'].iloc[0], "Tags for " + df_notes_002_better_tags['Term'].iloc[0])

assertEquals("wwwjdic" in df_notes_001_less_tags['tags'].iloc[0].split(), True, "Contains Tag 'wwwjdic'")
assertEquals("wwwjdic" in df_notes_002_better_tags['tags'].iloc[0].split(), False, "Contains Tag 'wwwjdic'")
assertEquals("fromdict" in df_notes_001_less_tags['tags'].iloc[0].split(), False, "Contains Tag 'fromdict'")
assertEquals("fromdict" in df_notes_002_better_tags['tags'].iloc[0].split(), True, "Contains Tag 'fromdict'")

---------------------------------------------------------------------------
Tags for 臨機応変
---------------------------------------------------------------------------
Before: wwwjdic yojijukugo
---------------------------------------------------------------------------
After: fromdict rare idiom
---------------------------------------------------------------------------


'OK'

In [103]:
df_notes_002_better_tags['tags'].value_counts()[:20]

                                       1621
fromdict                                796
fromtest textbook                       600
textbook textbook                       453
college textbook textbook               241
verb                                    200
fromdict verb                           144
fromexam                                126
len1                                    121
hiragana college textbook textbook      107
counter numeric                          97
numeric                                  81
addsimilar                               81
fromdict media                           72
college textbook semester1 textbook      71
fromexam textbook                        65
fromdict lyrics                          63
convo                                    61
n3 fromdict transitive verb verb         58
college textbook textbook katakana       54
Name: tags, dtype: int64

We can attempt to inspect which tags are most common, in which combinations, and which words would be ideal
for further additional metadata. However, **our tags are still lumped together** at this point. Also, there is
reason to believe that **some tags are showing up multiple times in the same tag string**. In order to properly count tag frequency, the duplicates must be confirmed absent (ie. found & removed). Then, the occurance (word frequency) of each tag may then be summed up for the tags column.

### 18. Inspect a note that you suspect has tag duplication

In [104]:
def inspect_note_by_id(df_in, nid):
    return df_in[df_in['nid']==nid]

In [105]:
# confirm that a particular note has tag duplicates
# crimison note id: 1369286386384
note_id_1 = 1369286386384
assertEquals(inspect_note_by_id(df_notes_002_better_tags,note_id_1).tags.values[0],"fromexam color fromexam len1","Four tags total with two duplicates exist")

'OK'

In [106]:
# example of item with tag duplication
sel2 = inspect_note_by_id(df_notes_002_better_tags,note_id_1)
sel2

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2
3847,1369286386384,1511481489,fromexam color fromexam len1,紅,くれない,<div>deep red; crimson</div><div><br /></div><...,,,,"<div>Common word, Noun</div><div>Common word, ...",...,,,,,,,,,,


### 19. Remove duplicate tags (convert tag strings > lists > sets > strings)

In [107]:
# Converts a tag string to a list to a set back to a string (this removes the duplicates)
def remove_dupes(t):
    temp = list(set(t.lower().split()))
    return ' '.join(temp) # return as string

In [108]:
df_notes_003_tags_no_dups = df_notes_002_better_tags.copy()
df_notes_003_tags_no_dups['tags'] = df_notes_003_tags_no_dups['tags'].apply(lambda x: remove_dupes(str(x)))

In [109]:
# determines if an individual tag substring exists in a larger tags list string
def tag_exists(tags, tag):
    return 1 if tag in tags.split() else 0

In [110]:
print(inspect_note_by_id(df_notes_003_tags_no_dups,note_id_1).tags.values[0])
assertEquals(tag_exists(inspect_note_by_id(df_notes_003_tags_no_dups,note_id_1).tags.values[0],"len1"), 1, "tag 'len1' remains")
assertEquals(tag_exists(inspect_note_by_id(df_notes_003_tags_no_dups,note_id_1).tags.values[0],"fromexam"), 1, "tag 'fromexam' remains")

len1 fromexam color


'OK'

It appears we have most, if not all, of the data we need to start. The format of the dates though is not yet human readable. Let's fix that.

### 20. Convert (& preserve) note ID to note creation date

In [111]:
#dueNum = 782 # this represents days from collection creation date
#crt = 1357635600 # this represents the collection creation date #todo: query dynamically from database
#print("mid 'model id': " + time.ctime(int("1768161991"))) # 1 day = 86400 seconds

df_notes_004_with_date = df_notes_003_tags_no_dups.copy()
df_notes_004_with_date['NoteCreated']= pd.to_datetime(df_notes_004_with_date['nid'],unit='ms')
df_notes_004_with_date['NoteCreated'] = df_notes_004_with_date['NoteCreated'].dt.date
df_notes_004_with_date.head()

print_before_after(df_notes_003_tags_no_dups['nid'].iloc[0], df_notes_004_with_date['NoteCreated'].iloc[0],"Term " + df_notes_004_with_date['Term'].iloc[0])

assertEquals(df_notes_004_with_date['nid'].iloc[0], 1331799797110, "Note ID is in Epoch Units")
assertEquals(str(df_notes_004_with_date['NoteCreated'].iloc[0]), "2012-03-15", "Note ID is in datetime date format year-month-day")

---------------------------------------------------------------------------
Term 臨機応変
---------------------------------------------------------------------------
Before: 1331799797110
---------------------------------------------------------------------------
After: 2012-03-15
---------------------------------------------------------------------------


'OK'

### 21. Generate Note Last Modified Date from "Mod" ID

In [112]:
df_notes_005_last_modified = df_notes_004_with_date.copy()
df_notes_005_last_modified['mod'] = pd.to_datetime(df_notes_005_last_modified['mod'],unit='s')
df_notes_005_last_modified['mod'] = df_notes_005_last_modified['mod'].dt.date

assertEquals(str(df_notes_005_last_modified['mod'].iloc[0]), "2017-11-23", "Note last modified is in datetime date format year-month-day")

'OK'

### 22. Create df_notes_final data frame for export & further usage

In [113]:
df_notes_final = df_notes_005_last_modified.copy()

### 23. Export df_notes_section_2_final

In [114]:
df_notes_final.to_csv('datasets/notes_section_2_final.csv')

### 24. Generate Card Creation Date from Card ID

In [115]:
df_cards_002_created_date = df_cards_001_less_cols.copy()
df_cards_002_created_date['CardCreated'] = pd.to_datetime(df_cards_002_created_date['id'],unit='ms')
df_cards_002_created_date['CardCreated'] = df_cards_002_created_date['CardCreated'].dt.date

assertEquals(str(df_cards_002_created_date['CardCreated'].iloc[0]), "2012-03-15", "Card ID is in datetime date format year-month-day")

'OK'

In [116]:
#queue           integer not null,
#      -- -3=sched buried, -2=user buried, -1=suspended,
#      -- 0=new, 1=learning, 2=due (as for type)

df_cards_003_no_new = df_cards_002_created_date.copy()
df_cards_003_no_new = df_cards_003_no_new[df_cards_003_no_new['queue']!=0] # remove cards marked as new
df_cards_003_no_new = df_cards_003_no_new[df_cards_003_no_new['reps']!=0] # remove cards that have not been reviewed
df_cards_003_no_new = df_cards_003_no_new[df_cards_003_no_new['queue']!=-1] # remove cards that are currently suspended
# https://stackoverflow.com/questions/18196203/how-to-conditionally-update-dataframe-column-in-pandas
df_cards_003_no_new.loc[df_cards_003_no_new['due'] > 10000, 'due'] = 0 # assign 0 to the due # todo: update w/ last studied date from revlog # todo: comment this line out once you have updated the collection import

print_before_after(df_cards_002_created_date.shape[0], df_cards_003_no_new.shape[0],"Card Rows:")

df_cards_003_no_new.tail(5)

---------------------------------------------------------------------------
Card Rows:
---------------------------------------------------------------------------
Before: 19315
---------------------------------------------------------------------------
After: 8245
---------------------------------------------------------------------------


,id,nid,ord,queue,due,ivl,factor,reps,lapses,CardCreated
18909,1549415185907,1549184119039,2,2,2388,87,2410,3,0,2019-02-06
18910,1550403009251,1550402953788,0,2,2232,1,2410,3,0,2019-02-17
18911,1550403009269,1550402953788,2,2,2302,1,2010,8,2,2019-02-17
18912,1550403084990,1550403040864,0,2,2383,82,2410,3,0,2019-02-17
18913,1550403085003,1550403040864,2,2,2240,5,2410,3,0,2019-02-17


In [117]:
# todo: add assertion to confirm that sel3 is empty (row size of 0)
# confirm that the three cards "in learning" have their due dates reset back to 0 for date transformation
sel3 = df_cards_003_no_new[df_cards_003_no_new['due'] == 0]
sel3.head()

,id,nid,ord,queue,due,ivl,factor,reps,lapses,CardCreated


### 25. Generate Due Date from Due Value

In [118]:
df_cards_004_due_date = df_cards_003_no_new.copy()
df_cards_004_due_date['due'] = pd_crt + df_cards_004_due_date['due'].map(timedelta)
df_cards_004_due_date['due'] = df_cards_004_due_date['due'].dt.date

assertEquals(str(df_cards_004_due_date['due'].iloc[0]), "2015-03-08", "Card due date is in datetime date format year-month-day")

'OK'

### 26. Create df_cards_final data frame for export & further usage

In [119]:
df_cards_final = df_cards_004_due_date.copy()

### 27. Export df_cards_section_2_final

In [120]:
df_cards_final.to_csv('datasets/cards_section_2_final.csv')

### 28. Merge card & note data frames to conduct cross analysis

In [121]:
# now that we have note id's for all the words, we can
# join together these separate dataframes
df_combo = pd.merge(df_notes_final, df_cards_final, on='nid')
print(df_combo.shape)
df_combo.head()

(8245, 66)


,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,NoteCreated,id,ord,queue,due,ivl,factor,reps,lapses,CardCreated
0,1331799797110,2017-11-23,rare fromdict idiom,臨機応変,りんきおうへん,adapting oneself to the requirements of the mo...,,,,"Noun, No-adjective",...,2012-03-15,1331799797110,0,2,2015-03-08,65,1680,10,1,2012-03-15
1,1331799797112,2017-11-23,fromdict,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",...,2012-03-15,1331799797112,0,2,2015-03-03,149,2080,8,1,2012-03-15
2,1331799797114,2017-11-23,fromdict,移籍,いせき,<div>changing household registry; transfer (e....,,,,"<div>Common word, Noun, Suru verb</div>",...,2012-03-15,1331799797114,0,2,2015-02-04,99,1980,7,0,2012-03-15
3,1331799797117,2017-11-23,fromdict verb,吊るす,つるす,to hang,,,,,...,2012-03-15,1331799797117,0,2,2015-03-17,143,2130,6,1,2012-03-15
4,1331799797118,2017-11-23,fromdict checked convo,和やか,なごやか,"harmonious, peaceful",,,,,...,2012-03-15,1331799797118,0,2,2015-02-06,74,1880,15,3,2012-03-15


It appears that card types are being rendered as numbers, which makes it less human readible. We will fix this. Additionally, our card model has a bunch of columns (fields) with no values in them, whatsoever. These can be taken out for the data analysis.

In [122]:
def is_blank (s):
    return not (s and s.strip())

In [123]:
def get_frame_of_cards_by_term(df, t):
    return df.loc[df['Term']==t]

In [124]:
# let's look a a small slice of data, to infer what we may
# we can take a broad overview look at the dataset to more quickly isolate candidates for removal
s = get_frame_of_cards_by_term(df_combo, '発明')
s.head()

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,NoteCreated,id,ord,queue,due,ivl,factor,reps,lapses,CardCreated
2899,1354094556789,2018-12-03,commonword fromtest suruverb noun textbook n3,発明,はつめい,"<span style=""""><div>invention</div></span>",,,,"Common word, Noun, Suru verb",...,2012-11-28,1354094556789,0,2,2015-06-30,80,1300,73,9,2012-11-28
2900,1354094556789,2018-12-03,commonword fromtest suruverb noun textbook n3,発明,はつめい,"<span style=""""><div>invention</div></span>",,,,"Common word, Noun, Suru verb",...,2012-11-28,1371807076626,4,2,2021-10-24,1056,1300,20,0,2013-06-21


### 29. Determine which columns (fields) are unused & can be safely removed

In [125]:
col_names = df_combo.columns.values
#print(is_blank(df_combo['Translation2'].iloc[0])) # see that this cell for this row is indeed blank

row_cnt = df_combo.shape[0] # number of rows in df_combo

# https://stackoverflow.com/questions/49677060/pandas-count-empty-strings-in-a-column
empty_strings = pd.DataFrame(df_combo.values == '',columns=col_names) # find all empty strings in a DataFrame
temp_dict = (empty_strings.sum()).to_dict()  # save the location of all empty strings as a DataFrame of booleans
removal_candidates = []
for key in temp_dict.items():
    if key[1] == row_cnt:
        removal_candidates.append(key[0])
print("Removal candidates:", removal_candidates)

Removal candidates: ['Sound3', 'AtoQ', 'AtoQaudio', 'AtoQkana', 'AtoQtranslation', 'QandApicture', 'answerPicture', 'blank1', 'blank2']


### 30. Trim unneeded (empty) columns from combo data frame

In [126]:
df_combo_001_less_cols = df_combo.copy()

removal_list = list(removal_candidates + ['queue'])

df_combo_001_less_cols = df_combo_001_less_cols.drop(removal_list,axis=1)

print_before_after(df_combo.shape, df_combo_001_less_cols.shape)
print_before_after(df_combo.columns.values, df_combo_001_less_cols.columns.values)

---------------------------------------------------------------------------
Before: (8245, 66)
---------------------------------------------------------------------------
After: (8245, 56)
---------------------------------------------------------------------------
---------------------------------------------------------------------------
Before: ['nid' 'mod' 'tags' 'Term' 'Yomi1' 'Translation' 'Translation2'
 'Translation3' 'AlternateForms' 'PartOfSpeech' 'Sound' 'Sound2' 'Sound3'
 'Examples' 'ExamplesAudio' 'AtoQ' 'AtoQaudio' 'AtoQkana'
 'AtoQtranslation' 'QandApicture' 'answerPicture' 'Meaning1'
 'SimilarWords' 'RelatedWords' 'Breakdown1' 'Comparison' 'Usage' 'Prompt1'
 'Prompt2' 'KakuMCD' 'IuMCD' 'ExtraMemo' 'Yomi2' 'Meaning2' 'Breakdown2'
 'Picture1' 'Picture2' 'Picture3' 'Picture4' 'HinshiMarker' 'Hint' 'Term2'
 'ArabicNumeral' 'CounterKanji' 'Mnemonic' 'SameSoundWords' 'Yomi3'
 'gChap' 'gBook' 'semester' 'gNumber' 'Transliteration' 'SoloLookCards'
 'TagOverflow' 'blank1' 'blank2

### 31. Label card types by their names

In [127]:
# ord stands for 'ordinal' : identifies which of the card templates it corresponds to
print(df_combo_001_less_cols['ord'].value_counts()) # these are the card vectors

# since our dataset contains a single card of a single card vector, & the card vectors
# aren't named/labeled, let's remove the outlier & add the names
df_combo_002_types_labeled = df_combo_001_less_cols.copy()
df_combo_002_types_labeled = df_combo_002_types_labeled.drop(df_combo_002_types_labeled[df_combo_002_types_labeled['ord'] == 11].index)

df_combo_002_types_labeled['ord'].value_counts() # the check shall pass

# now, to map the names onto the card vectors
df_combo_002_types_labeled['CardType'] = df_combo_002_types_labeled['ord'].map({0:'read', 2:'recall',4:'look',7:'listen'})
df_combo_002_types_labeled['CardType'].value_counts()

0     6843
4     1115
7      274
2       11
11       2
Name: ord, dtype: int64


read      6843
look      1115
listen     274
recall      11
Name: CardType, dtype: int64

### 32. Create binary exists/not columns based on presence of a given tag

In [128]:
def add_column_by_tag(df, tag):
    df[tag] = df['tags'].apply(lambda x: tag_exists(str(x), tag))

In [129]:
df_combo_003_with_binary = df_combo_002_types_labeled.copy()
inspect_list = ["commonword", "clothing", "animal", "body", "food", "place",
                "textbook", "college", "fromdict", "fromexam",
                "len1", "n1", "n2", "n3", "n4", "n5"
               ]
for item in inspect_list:
    add_column_by_tag(df_combo_003_with_binary, item)

In [130]:
df_combo_003_with_binary.dtypes.value_counts()

object    50
int64     23
dtype: int64

### 33. Create interval quartile sections for visualization purposes

In [131]:
# qcut: Quantile-based discretization function. Discretize variable into equal-sized buckets
# based on rank or based on sample quantiles. For example 1000 values for 10 quantiles would
# produce a Categorical object indicating quantile membership for each data point.
# http://www.datasciencemadesimple.com/quantile-decile-rank-column-pandas-python-2/
df_combo_003_with_binary['ivl_q'] = pd.qcut(df_combo_003_with_binary['ivl'],5,labels=False)
df_combo_003_with_binary.head()

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,college,fromdict,fromexam,len1,n1,n2,n3,n4,n5,ivl_q
0,1331799797110,2017-11-23,rare fromdict idiom,臨機応変,りんきおうへん,adapting oneself to the requirements of the mo...,,,,"Noun, No-adjective",...,0,1,0,0,0,0,0,0,0,0
1,1331799797112,2017-11-23,fromdict,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",...,0,1,0,0,0,0,0,0,0,1
2,1331799797114,2017-11-23,fromdict,移籍,いせき,<div>changing household registry; transfer (e....,,,,"<div>Common word, Noun, Suru verb</div>",...,0,1,0,0,0,0,0,0,0,0
3,1331799797117,2017-11-23,fromdict verb,吊るす,つるす,to hang,,,,,...,0,1,0,0,0,0,0,0,0,0
4,1331799797118,2017-11-23,fromdict checked convo,和やか,なごやか,"harmonious, peaceful",,,,,...,0,1,0,0,0,0,0,0,0,0


Let's further refine the dataframe entries to represent which notes have (1) visual data, (2) audio data, and (3) a L1 ("first language", English in this case) translation. We can represent these with binary values (0 for doesn't exist, 1 for exists).

### 35. Create boolean columns for predictive models

In [132]:
# Laura calls this process "Data Enriching"
intify_list = ['hasPOS','hasVisual','hasAudio','hasMultiMeaning','hasMultiReading','hasSimilar','hasHomophone','hasAltForm']

In [133]:
# https://stackoverflow.com/questions/17383094/how-can-i-map-true-false-to-1-0-in-a-pandas-dataframe
df_combo_003_with_binary['hasPOS'] = df_combo_003_with_binary['PartOfSpeech']!="" #todo: expand upon this, by tagify
df_combo_003_with_binary['hasVisual'] = df_combo_003_with_binary['Picture1']!=""
df_combo_003_with_binary['hasAudio'] = df_combo_003_with_binary['Sound']!=""
df_combo_003_with_binary['hasMultiMeaning'] = df_combo_003_with_binary['Translation2' and 'Translation3' and 'Meaning2']!=""
df_combo_003_with_binary['hasMultiReading'] = df_combo_003_with_binary['Yomi2']!="" # todo: inspect & incorporate venn diagram: https://commons.wikimedia.org/wiki/File:Homograph_homophone_venn_diagram.png
df_combo_003_with_binary['hasSimilar'] = df_combo_003_with_binary['SimilarWords']!=""
df_combo_003_with_binary['hasHomophone'] = df_combo_003_with_binary['SameSoundWords']!="" # write function, detect homophones
df_combo_003_with_binary['hasAltForm'] = df_combo_003_with_binary['Term2' and 'AlternateForms']!= ""

### 36. Drop non-numerical columns from combo data frame

In [134]:
df_combo_004_less_cols = df_combo_003_with_binary.copy()
df_combo_004_less_cols = df_combo_004_less_cols.drop(['Examples','ExamplesAudio',
                            'Meaning1','RelatedWords','Breakdown1','Comparison',
                           'Usage','Prompt1','Prompt2','KakuMCD','IuMCD','ExtraMemo',
                           'Breakdown2','Picture2','Picture3','Picture4','Mnemonic',
                            'Yomi3','gChap','gBook','semester','gNumber','ArabicNumeral',
                            'CounterKanji','SoloLookCards','HinshiMarker','Hint',
                            'mod','Transliteration'],axis=1)

In [135]:
# casts columns of type object to type int as directed, use with caution
def intify_bools(df, col):
    df[col] = df[col].astype(int)

### 37. Ensure numerical/boolean types are encoded properly

In [136]:
df_combo_004_less_cols.dtypes.value_counts()

int64     24
object    21
bool       8
dtype: int64

In [137]:
for item in intify_list:
    intify_bools(df_combo_004_less_cols,item)

In [138]:
df_combo_004_less_cols.dtypes.value_counts()

int64     32
object    21
dtype: int64

### 38. Further reduce columns not in use 

In [139]:
df_combo_004_less_cols = df_combo_004_less_cols.drop(['Picture1','Sound','Sound2',
                              'TagOverflow','Translation2', 'Meaning2','Yomi2','Term2',
                              'SameSoundWords','hasPOS','SimilarWords','AlternateForms',
                            'Translation3'],axis=1)
df_combo_004_less_cols.head(35)[30:]

#selection2 = df_binary.loc[df_binary['hasMultiMeaning']==1]
#selection2.head()

,nid,tags,Term,Yomi1,Translation,PartOfSpeech,NoteCreated,id,ord,due,...,n4,n5,ivl_q,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm
30,1342506824723,media len1 ghibli textbook suffix magazine,者,もの,"<span style="""">person (rarely used w.o. a qual...","<span style="""">Common word, Noun</span><div>No...",2012-07-17,1342506824723,0,2016-12-01,...,0,0,4,0,0,0,0,0,0,0
31,1342506824724,fromdict,事件,じけん,event; affair; incident; case; plot; trouble; ...,"Common word, Noun",2012-07-17,1342506824724,0,2015-05-07,...,0,0,2,0,0,0,0,0,0,0
32,1342506824725,fromdict,事情,じじょう,circumstances; consideration; conditions; situ...,"Common word, Noun",2012-07-17,1342506824725,0,2015-07-10,...,0,0,2,0,0,0,0,0,0,0
33,1342506824726,fromdict,事柄,ことがら,matter; thing; affair; circumstance,"Common word, Noun",2012-07-17,1342506824726,0,2015-11-10,...,0,0,2,0,0,0,0,1,0,0
34,1342506824727,fromdict,物事,ものごと,things; everything,"Common word, Noun",2012-07-17,1342506824727,0,2016-03-17,...,0,0,2,0,0,0,0,1,0,0


### 39. Count syllable count & character length for each term

In [140]:
df_combo_005_with_len = df_combo_004_less_cols.copy()

df_combo_005_with_len['TermLen'] = df_combo_005_with_len['Term'].str.len()
df_combo_005_with_len['Syllables'] = df_combo_005_with_len['Yomi1'].str.len()
df_combo_005_with_len.loc[df_combo_005_with_len['Syllables'] == 0, 'Syllables'] = df_combo_005_with_len['TermLen']

bins = [0,1,2,4,8,128]
labels = ["[1]","[2]","[3:4]","[5:8]","[9: ]"]
# https://stackoverflow.com/questions/45273731/binning-column-with-python-pandas
df_combo_005_with_len['TermLenGroup'] = pd.cut(df_combo_005_with_len['TermLen'], bins=bins, labels=labels)

#df.loc[df['Grades'] <= 77, 'Grades'] = 100
# https://stackoverflow.com/questions/42815768/pandas-adding-column-with-the-length-of-other-column-as-value
#df_binary2.head(35)[30:]
df_combo_005_with_len.tail(20)[:10]

,nid,tags,Term,Yomi1,Translation,PartOfSpeech,NoteCreated,id,ord,due,...,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,TermLen,Syllables,TermLenGroup
8225,1485705402623,commonword clothing noun gairaigo katakana,ブラジャー,,1. bra; brassiere,,2017-01-29,1485707774754,4,2019-05-02,...,1,0,0,0,0,0,0,5,5,[5:8]
8226,1489373157595,,細切り,ほそぎり,thin strips; matchstick-like strips; julienned...,Noun,2017-03-13,1489373228966,0,2017-03-25,...,0,0,0,0,0,0,0,3,4,[3:4]
8227,1489756408272,,離陸,りりく,take-off (getting off the ground),,2017-03-17,1489756526501,0,2017-03-18,...,0,0,0,0,0,0,0,2,3,[2]
8228,1508007342589,phrases basics commonword kana,どういたしまして,,you are welcome; don't mention it; not at all;...,,2017-10-14,1508007536099,0,2018-07-19,...,1,1,0,0,0,0,0,8,8,[5:8]
8229,1508007342589,phrases basics commonword kana,どういたしまして,,you are welcome; don't mention it; not at all;...,,2017-10-14,1508007536100,7,2018-08-16,...,1,1,0,0,0,0,0,8,8,[5:8]
8230,1508007536188,phrase basics formal polite question,お名前は何ですか？,おなまえ は なん ですか？,What is your name?,,2017-10-14,1508007825484,7,2018-06-18,...,0,1,0,0,0,0,0,9,14,[9: ]
8231,1508012628565,basics n5 kana commonword,いいえ,,no,,2017-10-14,1508013224329,0,2018-03-24,...,1,1,0,0,0,0,0,3,3,[3:4]
8232,1508012628565,basics n5 kana commonword,いいえ,,no,,2017-10-14,1511480991113,2,2019-05-02,...,1,1,0,0,0,0,0,3,3,[3:4]
8233,1508012628565,basics n5 kana commonword,いいえ,,no,,2017-10-14,1511480991114,4,2018-03-22,...,1,1,0,0,0,0,0,3,3,[3:4]
8234,1508012628565,basics n5 kana commonword,いいえ,,no,,2017-10-14,1511480991115,7,2018-03-22,...,1,1,0,0,0,0,0,3,3,[3:4]


In [141]:
# inspect the many syllable entries
df_many_syl = df_combo_005_with_len.copy()
many_syl = df_many_syl['Syllables'] > 20
df_many_syl.loc[many_syl]

,nid,tags,Term,Yomi1,Translation,PartOfSpeech,NoteCreated,id,ord,due,...,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,TermLen,Syllables,TermLenGroup
247,1346057958628,fromdict,東京電力福島・第１原発事故,とうきょうでんりょくふくしま・だいいちげんぱつじこ,,,2012-08-27,1346057958628,0,2015-07-07,...,0,0,0,0,0,0,0,13,25,[9: ]
316,1346215143756,datesandtime fromdict numeric,1837～1901年,せんはっぴゃくさんじゅうななねんからせんきゅうひゃくいちねん,,,2012-08-29,1346215143756,0,2015-07-14,...,0,0,0,0,0,0,0,10,30,[9: ]


In [143]:
df_combo_005_with_len.columns.values

array(['nid', 'tags', 'Term', 'Yomi1', 'Translation', 'PartOfSpeech',
       'NoteCreated', 'id', 'ord', 'due', 'ivl', 'factor', 'reps',
       'lapses', 'CardCreated', 'CardType', 'commonword', 'clothing',
       'animal', 'body', 'food', 'place', 'textbook', 'college',
       'fromdict', 'fromexam', 'len1', 'n1', 'n2', 'n3', 'n4', 'n5',
       'ivl_q', 'hasVisual', 'hasAudio', 'hasMultiMeaning',
       'hasMultiReading', 'hasSimilar', 'hasHomophone', 'hasAltForm',
       'TermLen', 'Syllables', 'TermLenGroup'], dtype=object)